In [12]:
%%writefile ./Query.py
import yad
import argparse

import requests
from bs4 import BeautifulSoup
import sys,re,goslate
from translate import Translator
from unidecode import unidecode as remove_accents
translator= Translator(from_lang="es",to_lang="de")


def es_to_de(string):
    return translator.translate(remove_accents(string))

import subprocess


#OTHER PYTHON FILES IN SAME DIR
from add_card import add_image_card
from linguee_query import ask,extract_info
from yad_args import yad_args, yad_img_args

yad=yad.YAD()

bash_script_path="~/python_scripts/anki_scripts/"


def get_soup_object(url):
	return BeautifulSoup(requests.get(url).text)


def synonyms(word):
    url="http://www.wordreference.com/sinonimos/%s"%(word)
    soup=get_soup_object(url)
    try:
        results=soup.find_all(class_="trans clickable")[0].find_all("li")[0]
        return results.text
    except:
        print("could not find synonyms :(")
        return ""

def html_list(str_list):
    start="<ul>\n"
    end="</ul>\n"
    middle=["<li type=\"square\">"+item+"</li>\n" for item in str_list]
    ret_str=start+"".join(middle)+end
    return ret_str



class Query:

    def __init__(self, search_term,
                 Type="",
                 gender="",
                 response=[],
                 audio_link="",
                 audio_base_url="http://www.linguee.de/mp3/%s.mp3",
                 examples=[],#["Mi gato es negro.","Mi gato es verde."],
                 linguee_query_url="https://linguee-api.herokuapp.com/api?q=%s&src=es&dst=%s",
                 synonyms=[],
                 anki_user="new_user",
                 output_path="/home/david/gen_ank/"
                ):
        self.search_term = search_term.strip().lower()
        self.folder=self.search_term.replace(" ","_")
        self.type = Type
        self.gender=gender
        self.response = response
        self.audio_link=audio_link
        self.examples=examples
        self.audio_base_url=audio_base_url
        self.linguee_query_url=linguee_query_url
        self.synonyms=synonyms
        self.front=self.search_term
        self.anki_user=anki_user
        self.output_path=output_path
        self.translated=""
        self.trans_syns=[]
        
    def translate(self):
        translations=[x["translations"] for x in self.response["exact_matches"]]
        #print(translations)
        translations=[[x["text"] for x in y] for y in translations]
        translation_string="\n".join([", ".join(x) for x in translations])
        print(translation_string)
        self.translated=translation_string
        #print(self.translated)
        #self.trans_syns=[es_to_de(syn) for syn in self.synonyms]
        #print(self.translated,self.transl_syn)

    def linguee_query(self):
        self.word_or_phrase()
        qry_str=self.search_term.replace(" ","+")
        #print(qry_str)
        self.response=ask(qry_str,self.linguee_query_url)
        print(extract_info(self.response))
        #print(ask(qry_str))
        if self.type=="phrase":
            #print("phrase",qry_str)
            self.audio_link,b,c,self.examples=extract_info(self.response)
        else:
            self.audio_link,self.word_type,self.gender,self.examples=extract_info(self.response)
        #response=html_response
        #change languages
        #yad: Let user know if not possible to connect
        #yad: Let user know if no matches
    
    def word_or_phrase(self):
        #single word or phrase?
        if " " in self.search_term:
            #print("SPACE")
            self.type="phrase"

    def print_all(self):
        print("-----------------------------------------------")
        print("Search term: %s"%self.search_term)
        print("Folder: %s"%self.folder)
        print("Type : %s"%self.type)
        print("Synonyms: %s"%self.synonyms)
        print("Audio Link: %s"%self.audio_link)
        print("Example:%s"%self.examples)
        print("-----------------------------------------------")


    def download_images(self):
        print("Downloading images...")
        process=subprocess.Popen([bash_script_path+"my_image_download %s"%self.search_term],shell=True)
        return process

    def set_synonyms(self):
        self.synonyms=synonyms(self.search_term).split(",")
        self.synonyms=[syn for syn in self.synonyms if syn != ""]
        self.trans_syns=["" for syn in self.synonyms]
        
    
    def download_audio(self):
        audio_url=self.audio_base_url%self.audio_link
        print("Downloading audio-file...")
        process=subprocess.Popen([bash_script_path+"my_audio_download %s %s"%(self.folder,audio_url)],shell=True)
        #print("Download of audio-file finished.")
        return process
        
    
    def check_attributes(self):
        #starts yad to let user check stuff
        field_list=yad_args(path=self.output_path,
                                search_term=self.search_term,
                                translated=self.translated,
                                folder=self.folder,
                                #word_type=self.type,
                                gender=self.gender,
                                synonyms=self.synonyms,
                                #trans_syns=self.trans_syns,
                                #images=["1.jpg","2.jpg","3.jpg"],
                                examples=self.examples)
        for x in field_list:
            print(x)
        user_input=yad.Form(center=1,on_top=1,fields=field_list)
        
        if user_input==None:
            #return False
            print("Abbruch")
            #exit()
        elif user_input["rc"]==0:
            user_input.pop("rc")
            #rint(user_input)
        else:
            print("Abbruch")
            #return False
                    #exit()
        #print(user_input)
        #change mistakes
        if user_input[0]!="-":
            self.front=user_input[0]+" "+user_input[1]
        else:
            self.front=user_input[1]
        sl=len(self.synonyms)
        el=len(self.examples)
        for i in reversed(range(sl)):
            #print(i+3)
            if user_input[i+2]=="FALSE":
                del self.synonyms[i]
        for i in reversed(range(el)):
            if user_input[i+2+sl]=="FALSE":
                del self.examples[i]
                
    def choose_image(self):
        yad.Form(center=1,on_top=1,
                            fields=yad_img_args(path=self.output_path,folder=self.folder,images=["1.jpg","2.jpg","3.jpg"]))

            #print(i+3+sl)

        #{0: 'el', 1: 'gato negro', 2: 'noun', 3: 'FALSE', 4: 'FALSE',
        # 5: 'FALSE', 6: 'FALSE', 7: 'FALSE', 8: 'FALSE', 9: 'FALSE', 10: '', 11: '', 12: ''}
    
    def mark_examples(self):
        for word in self.search_term.split(" "):
            self.examples=[re.sub(r'((?i)%s)'%word,r'<font color=red><b>\1</font></b>',ex) for ex in self.examples]
    
    def generate_card(self):
        syn_str=", ".join(self.synonyms)
        ex_str=html_list(self.examples)
        add_image_card(word=self.front,
                       file=self.folder,
                       synonyms=syn_str,
                       examples=ex_str)
                           
    
    def get_all_data(self):
        images_process=self.download_images()
        self.set_synonyms()
        #self.translate()
        try:
            print("trying..."+self.linguee_query_url)
            self.linguee_query()
        except:
        #    print("exception")
            print("trying instead: "+"http://localhost:8000/api?q=%s&src=es&dst=%s")
            self.set_api_url("http://localhost:8000/api?q=%s&src=es&dst=%s")
            self.linguee_query()
        try:
            self.translate()
        except:
            print("Could not find translation.")
        audio_process=self.download_audio()
        self.print_all()
        self.check_attributes()
        images_process.wait()
        self.choose_image()
        self.mark_examples()
        audio_process.wait()
        
    def import_card_to_deck(self):
        print("Importing Card to Deck")
        subprocess.call(["bash -c \" timeout 3 anki -p %s %s/output.apkg \" "%(self.anki_user,self.output_path)],shell=True)
        print("Finished")
        #print("Download finished.")
    
        
    def set_api_url(self,url):
        self.linguee_query_url=url
    
    
    
        
def main(search_term,api_url):
    cat=Query(search_term,
              linguee_query_url=api_url)
              #linguee_query_url="http://localhost:8000/api?q=%s&src=es&dst=%s")
    cat.get_all_data()
    cat.generate_card()
    cat.import_card_to_deck()
#cat.generate_card()

#cat.set_synonyms()

#cat.print_all()
#cat.download_images()
#cat.download_audio()
#cat.check_attributes(path)
#cat.check_attributes(path)
#cat.check_attributes(path)

if __name__ == "__main__":
    
    parser = argparse.ArgumentParser(description="Adds Anki card to test_deck")

    parser.add_argument('--word', type=str, help="the search word")
    parser.add_argument('--api', type=str, help="the url of the api")
    

    args = parser.parse_args()
    search_term=args.word
    if args.api==None:
        api_url="https://linguee-api.herokuapp.com/api?q=%s&src=es&dst=%s"
    else:
        api_url=args.api

   # stuff only to run when not called via 'import' here
    main(search_term,api_url)


Overwriting ./Query.py


In [ ]:
#%run Query.py --word="seguir"
import Query

new_query=Query.Query("llegada")
new_query.get_all_data()
#new_query.print_all()

#new_query.get_all_data()
#print(new_query.translated)


trying...https://linguee-api.herokuapp.com/api?q=%s&src=es&dst=%s
querying ... (Language: de)
200
('ES_ES/06/062118dc95f315ccf7200a4b1f17e52e-107', 'noun', 'f', ['Hoy terminan los largos días de preparación y me siento cerca, [...] muy cerca de la llegada del Padre mío [...] y Padre de todos los hombres.', 'La enorme llegada de jóvenes habilitados en el uso de Internet que se graduaron en escuelas del mundo en desarrollo tuvo algunos efectos inesperados.'])
Ankunft, Eintreffen, Anreise, Aufkommen
erreichen, werden, kommen, gelangen, ankommen, zu jdm. kommen, eintreffen, zu etw.Dat gelangen
-----------------------------------------------
Search term: llegada
Folder: llegada
Type : 
Synonyms: ['arribada', ' venida', ' aparición', ' presencia', ' presentación', ' comparecencia']
Audio Link: ES_ES/06/062118dc95f315ccf7200a4b1f17e52e-107
Example:['Hoy terminan los largos días de preparación y me siento cerca, [...] muy cerca de la llegada del Padre mío [...] y Padre de todos los hombres.', 

Ankunft, Eintreffen, Anreise, Aufkommen
erreichen, werden, kommen, gelangen, ankommen, zu jdm. kommen, eintreffen, zu etw.Dat gelangen


In [11]:
%%writefile yad_args.py


def yad_args(path,search_term,translated,folder,gender,synonyms,examples):#,images):
    #front=""
    #print(gender)
    article_list=["el","la"," "]
    if gender=="m":
        pass
    elif gender=="f":
        #print("FEMININ")
        article_list[0],article_list[1]=article_list[1],article_list[0]
    else:
        article_list[0],article_list[2]=article_list[2],article_list[0]
        #article_list[2],article_list[0]=article_list[0],article_list[2]
        
    main_buttons=[
        ["LBL","Meaning",translated],
        ["CB","Article",article_list],
        ["","Word or Phrase:",search_term]]
        #["","Type:",word_type]]
        
    #for i in range(len(synonyms)):
    #    synonyms[i]=synonyms[i]+"  -  "+trans_syns[i]
    #print(main_buttons)
    syn_buttons = [["CHK","Synonym: "+syn,"false"] for syn in synonyms]
    
    example_buttons=[["CHK","Example: "+ex,"true"] for ex in examples]    
    #for i in im_buttons:
        #print(i)
    return main_buttons+syn_buttons+example_buttons#+im_buttons

def yad_img_args(path,folder,images):
    im_buttons=[["BTN",['',path+folder+"/"+im],
                 "bash -c \"cd %s; cp \"%s\" %s.jpg;timeout 1 xdotool key \"Escape\" \""%(path,folder+"/"+im,folder)]for im in images]
    return im_buttons






Overwriting yad_args.py


In [11]:
%%writefile ./linguee_query.py
import warnings
warnings.filterwarnings('ignore')
import requests
import itertools

languages=itertools.cycle(("de","en","es","pt","it","fr","el"))
#languages=itertools.cycle(("en","el","es","de","pt","it"))
#itertools.cycle(("en", "el", "es","bg", "cs", "da","de", "et", "fi", "fr",
          #                     "hu", "it", "ja", "lt", "lv", "mt", "nl", "pl", "pt", "ro",
          #                     "ru", "sk", "sl", "sv", "zh"))

#linguee_api_url="https://linguee-api.herokuapp.com/api?q=%s&src=es&dst=%s"




def ask_once(qry_str,lang,try_no,linguee_api_url):
    print("querying ... (Language: %s)"%lang)
    data=requests.get(linguee_api_url%(qry_str,lang))
    print(data.status_code)
    #print(data.json())
    if data.status_code==200:
        #print(data.json())
        if data.json()["exact_matches"]==None:
            lang=next(languages)
            return ask_once(qry_str,lang,try_no+1,linguee_api_url)
        return data.json()
    if data.status_code==500:
        lang=next(languages)
        if try_no==20:
            return {"exact_matches":None}
        else:
            return ask_once(qry_str,lang,try_no+1,linguee_api_url)
        
        
def extract_info(response):
    if response["exact_matches"]!=None:
        examples=[]
        match=response["exact_matches"][0]
        audio_id=match["audio_links"][0]["url_part"]
        word_type=match["word_type"]["pos"]
        if word_type=="noun":
            gender=match["word_type"]["gender"][0]
        else:
            gender=""
        examples=[]
        real_ex=[x["src"] for x in response["real_examples"]]#.sort(key = lambda s: len(s))
        for key in match["translations"]:
            try:
                examples.append(key["examples"][0]["source"])
            except:
                pass
        examples=list(set(examples))
        i=len(examples)
        for x in real_ex:
            if(len(x)<150):
                examples.append(x)
                i+=1
            if i==3:
                break
        return audio_id,word_type,gender,examples
    
        
def ask(qry_str,linguee_api_url):
    languages=itertools.cycle(("de","en","es","pt","it","fr","el"))
    return ask_once(qry_str,"de",0,linguee_api_url)
        





Overwriting ./linguee_query.py


In [25]:
%%writefile add_card.py
import genanki
import os

path="/home/david/gen_ank/"

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)



#parser = argparse.ArgumentParser(description="Adds Anki card to test_deck")

#parser.add_argument('--word', type=str, help="the search word")
#parser.add_argument("--model",type=str, help="model ")
#parser.add_argument("--syn",type=str, help="model ")


#args = parser.parse_args()
#word=args.word
#model=args.model
#syn=args.syn





img_model = genanki.Model(
  1607392319,
  'Image',
  fields=[
    {'name': 'Word'},
    {'name': 'Sound'},
    {'name': 'Picture'},
    {'name': 'Synonyms'},
    {'name': 'Example'}
  ],
  templates=[
    {
      'name': 'ES-IMG',
      'qfmt': '<font size="7" face="arial"> <center><b>{{Word}}</b> {{Sound}}</font>',
      'afmt': """{{FrontSide}}
                <hr id="answer">
                <font size="4">
                <font color="blue"> <b>Sinónimo(s): </b> {{Synonyms}} </font><br />
                <br />
                {{Picture}}<br />
                <b> Ejemplo(s):</b><br /> <em>{{Example}}</em>""",

    },
    {
      'name': 'IMG-ES',
      'qfmt': """<center>{{Picture}}<br />
                <font color="blue" size="4"> <b>Sinónimo(s): </b> {{Synonyms}} </font><br />
                <br />""",
        #'<center>{{Picture}}<br />{{Synonyms}}<br />{{Example}}',
      'afmt': """{{FrontSide}}
                <hr id="answer">
                <font size="7" face="arial"> <center><b>{{Word}}</b> {{Sound}}</font><br />
                <font size="4">
                <b> Ejemplo(s):</b><br /> <em>{{Example}}</em></font>
                """,
        #'{{FrontSide}}<hr id="answer"><font size="6" face="arial"> <center><b>{{Word}}</b> {{Sound}}</font>',
    },
  ])


cloze_model = genanki.Model(
  1607392320,
  'Lückentext',fields=[{"name":"Text",
                       "name":"Extra"}],templates=[{"name":"Lückentext"}])


syn_model = genanki.Model(
  1607392318,
  'Synonyms',
  fields=[
    {'name': 'Word'},
    {'name': 'Sound'},
    {'name': 'Synonyms'}
  ],
  templates=[
    {
      'name': 'ES-Syn',
      'qfmt': '{{Word}} {{Sound}}',
      'afmt': '{{FrontSide}}<hr id="answer"> {{Synonyms}}',
    },
   
  ])



def add_image_card(word,file,synonyms,examples):
    first_deck=genanki.Deck(1,"test_deck")
    my_package=genanki.Package(first_deck)
    with cd(path):
        if os.path.isfile("%s.mp3"%file):
            my_package.media_files.append("%s.mp3"%file)
            print("added %s.mp3"%file)
        else:
            print("Could not find Sound-File :(")
        if os.path.isfile("%s.jpg"%file):
            my_package.media_files.append("%s.jpg"%file)
            print("added %s.jpg"%file)

        else:
            print("Could not find Imgage-File  :(")
        my_note=genanki.Note(model=img_model,fields=[word,
                                                     "[sound:%s.mp3]"%file,
                                                     '<img src="%s.jpg">'%file,
                                                     synonyms,
                                                     examples])

        first_deck.add_note(my_note)
        my_package.write_to_file("output.apkg")
        
def syn_card():
    my_note=genanki.Note(
        model=syn_model,
        fields=[word,"[sound:%s.mp3]"%word,syn])

def add_cloze_card(cloze_phrase):
    first_deck=genanki.Deck(1,"test_deck")
    my_package=genanki.Package(first_deck)
    my_note=genanki.Note(model=cloze_model,fields=[cloze_phrase])
    first_deck.add_note(my_note)
    my_package.write_to_file("output.apkg")
    
#first_deck.add_note(my_note)


#add_image_card("word","casarse","synonym","examples")

#my_package.write_to_file("output.apkg")


Overwriting add_card.py


In [10]:
import yad
new=yad.YAD()

?new.Form